In [3]:
# Cellule d'installation des bibliothèques
!pip install tensorflow opencv-python-headless matplotlib numpy scikit-learn -q
!pip install mlflow -q

In [5]:
# Accéder au secret
from google.colab import userdata
import os

GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_USER = 'M-Carre'
GITHUB_REPO_NAME = 'OC-autonomous-vehicle-segmentation'
CLONE_DIR = f"/content/{GITHUB_REPO_NAME}"

# Construire l'URL de clonage
clone_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{GITHUB_REPO_NAME}.git"

# Exécuter la commande git clone
# os.system() est parfois plus propre pour ça que ! avec f-strings complexes
os.system(f"git clone {clone_url} {CLONE_DIR}")

# Vérifier
if os.path.exists(CLONE_DIR):
    %cd {CLONE_DIR}
    !ls
    print(f"Dépôt cloné avec succès dans {CLONE_DIR}")
else:
    print(f"Échec du clonage du dépôt dans {CLONE_DIR}")

/content/OC-autonomous-vehicle-segmentation
README.md
Dépôt cloné avec succès dans /content/OC-autonomous-vehicle-segmentation


# Script GitHub

In [14]:
# ==============================================================================
# SCRIPT DE SYNCHRONISATION (DRIVE -> REPO LOCAL) ET PUSH VERS GITHUB (V2)
# ==============================================================================
import os
import shutil
from google.colab import userdata

# --- Configuration des Chemins et Fichiers ---
PROJECT_BASE_ON_DRIVE = '/content/drive/MyDrive/Colab Notebooks/OC/P8' # ADAPTEZ CECI
GIT_REPO_PATH_IN_COLAB = '/content/OC-autonomous-vehicle-segmentation'
GIT_BRANCH_NAME = 'main'
ITEMS_TO_SYNC = [
    "notebooks/Notebook_modélisation.ipynb",
    "src",
    # "scripts", # Décommentez si vous l'avez créé sur Drive
    ".gitignore",
    "README.md",
    "api",
    "webapp",
    "reports",
    "models",
]
# ------------------------------------------------------------------------------

def sync_and_push_to_github_v2():
    print("### 1. VÉRIFICATION DES CHEMINS ###")
    if not os.path.exists(PROJECT_BASE_ON_DRIVE):
        print(f"ERREUR : Le dossier source sur Drive '{PROJECT_BASE_ON_DRIVE}' n'existe pas.")
        return
    if not os.path.exists(GIT_REPO_PATH_IN_COLAB):
        print(f"ERREUR : Le dossier du dépôt Git cloné '{GIT_REPO_PATH_IN_COLAB}' n'existe pas.")
        return
    print(f"Source du projet sur Drive : {PROJECT_BASE_ON_DRIVE}")
    print(f"Dépôt Git local dans Colab : {GIT_REPO_PATH_IN_COLAB}")

    print("\n### 2. NAVIGATION VERS LE RÉPERTOIRE DU DÉPÔT GIT LOCAL ###")
    try:
        os.chdir(GIT_REPO_PATH_IN_COLAB)
        print(f"Changé de répertoire vers : {os.getcwd()}")
    except FileNotFoundError:
        print(f"ERREUR CRITIQUE : Impossible de naviguer vers le répertoire du dépôt Git '{GIT_REPO_PATH_IN_COLAB}'.")
        return

    print("\n### 3. MISE À JOUR DEPUIS LE DÉPÔT DISTANT (GIT PULL) ###")
    # Configurez l'identité Git AVANT toute opération Git si ce n'est pas déjà fait dans la session
    # print("Configuration de l'identité Git (si nécessaire)...")
    !git config --global user.email "carre.mathis@proton.me"
    !git config --global user.name "M-Carre"

    print(f"Tentative de 'git pull origin {GIT_BRANCH_NAME}'...")
    pull_result = os.system(f"git pull origin {GIT_BRANCH_NAME}") # Utiliser os.system pour mieux voir les erreurs potentielles de pull
    if pull_result != 0:
        print(f"ERREUR : 'git pull' a échoué avec le code {pull_result}. Veuillez vérifier les messages ci-dessus.")
        print("Résolvez les conflits ou problèmes de pull avant de continuer.")
        # Il pourrait être judicieux d'arrêter le script ici si le pull échoue gravement
        # return
    else:
        print("'git pull' terminé avec succès (ou pas de changements distants).")


    print("\n### 4. SYNCHRONISATION DES FICHIERS (DRIVE -> REPO LOCAL) ###")
    synced_item_count = 0
    for item_rel_path in ITEMS_TO_SYNC:
        source_item_abs_path = os.path.join(PROJECT_BASE_ON_DRIVE, item_rel_path)
        # La destination est relative au dossier courant (qui est GIT_REPO_PATH_IN_COLAB)
        dest_item_abs_path = os.path.join(os.getcwd(), item_rel_path) # ou juste item_rel_path

        if not os.path.exists(source_item_abs_path):
            print(f"AVERTISSEMENT : L'élément source '{source_item_abs_path}' n'existe pas sur Drive. Ignoré.")
            continue

        os.makedirs(os.path.dirname(dest_item_abs_path), exist_ok=True)

        if os.path.isdir(source_item_abs_path):
            if os.path.exists(dest_item_abs_path):
                print(f"   Suppression du dossier de destination existant : {dest_item_abs_path}")
                shutil.rmtree(dest_item_abs_path)
            print(f"   Copie du dossier : '{source_item_abs_path}' vers '{dest_item_abs_path}'")
            shutil.copytree(source_item_abs_path, dest_item_abs_path)
            synced_item_count += 1
        elif os.path.isfile(source_item_abs_path):
            print(f"   Copie du fichier : '{source_item_abs_path}' vers '{dest_item_abs_path}'")
            shutil.copy2(source_item_abs_path, dest_item_abs_path)
            synced_item_count += 1
    if synced_item_count > 0: print(f"{synced_item_count} élément(s) synchronisé(s) depuis Drive.")


    print("\n### 5. OPÉRATIONS GIT (ADD, COMMIT, PUSH) ###")
    print("\n--- Statut Git (après synchronisation et pull) ---")
    !git status

    print("\nAjout de tous les changements au staging area (git add .)...")
    !git add .

    # Vérifier à nouveau le statut après add pour voir ce qui sera commité
    print("\n--- Statut Git (après 'git add .') ---")
    !git status --short # Version courte pour voir les fichiers staged

    commit_message = input("\nEntrez votre message de commit (laissez vide pour annuler le commit/push) : ")

    if not commit_message:
        print("\nMessage de commit vide. Opération de commit et push annulée.")
        print("\n--- Statut Git Actuel (changements non commités) ---")
        !git status
        return

    print(f"\nCommit des changements avec le message : '{commit_message}'")
    os.environ['GIT_COMMIT_MESSAGE'] = commit_message
    !git commit -m "$GIT_COMMIT_MESSAGE"
    # !git commit -m "{commit_message}" # Moins robuste si le message a des caractères spéciaux
    del os.environ['GIT_COMMIT_MESSAGE']

    print(f"\nPush des changements vers la branche '{GIT_BRANCH_NAME}' sur GitHub...")
    push_result = os.system(f"git push origin {GIT_BRANCH_NAME}")
    if push_result == 0:
        print("\n--- Opération de synchronisation, commit et push terminée avec succès. ---")
    else:
        print(f"ERREUR : 'git push' a échoué avec le code {push_result}. Vérifiez les messages.")

    print("Vérifiez votre dépôt sur GitHub pour confirmer les changements.")

# --- Exécution de la fonction ---
sync_and_push_to_github_v2()

### 1. VÉRIFICATION DES CHEMINS ###
Source du projet sur Drive : /content/drive/MyDrive/Colab Notebooks/OC/P8
Dépôt Git local dans Colab : /content/OC-autonomous-vehicle-segmentation

### 2. SYNCHRONISATION DES FICHIERS (DRIVE -> REPO LOCAL) ###
   Copie du fichier : '/content/drive/MyDrive/Colab Notebooks/OC/P8/notebooks/Notebook_modélisation.ipynb' vers '/content/OC-autonomous-vehicle-segmentation/notebooks/Notebook_modélisation.ipynb'
   Suppression du dossier de destination existant : /content/OC-autonomous-vehicle-segmentation/src
   Copie du dossier : '/content/drive/MyDrive/Colab Notebooks/OC/P8/src' vers '/content/OC-autonomous-vehicle-segmentation/src'
AVERTISSEMENT : L'élément source '/content/drive/MyDrive/Colab Notebooks/OC/P8/scripts' n'existe pas sur Drive. Ignoré.
AVERTISSEMENT : L'élément source '/content/drive/MyDrive/Colab Notebooks/OC/P8/.gitignore' n'existe pas sur Drive. Ignoré.
AVERTISSEMENT : L'élément source '/content/drive/MyDrive/Colab Notebooks/OC/P8/README

In [15]:
%cd /content/OC-autonomous-vehicle-segmentation
!git pull origin main # ou votre branche

/content/OC-autonomous-vehicle-segmentation
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 2.51 KiB | 1.25 MiB/s, done.
From https://github.com/M-Carre/OC-autonomous-vehicle-segmentation
 * branch            main       -> FETCH_HEAD
   cbd8a94..32d5add  main       -> origin/main
hint: You have divergent branches and need to specify how to reconcile them.
hint: You can do so by running one of the following commands sometime before
hint: your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-

In [16]:
%cd /content/OC-autonomous-vehicle-segmentation
!git pull --rebase origin main

/content/OC-autonomous-vehicle-segmentation
From https://github.com/M-Carre/OC-autonomous-vehicle-segmentation
 * branch            main       -> FETCH_HEAD
Successfully rebased and updated refs/heads/main.
